# Analyze Text

 :::{admonition} Development Process
 :class: important
- Come up with an idea 😀
- Identify a dataset and specify precisely what you want to extract
- Engineer a prompt (use OpenAI Playground)
- Evaluate performance on a sample
    * If performance is unacceptable, try further prompt engineering, functions, etc.
    * If performance is still not good enough, try fine-tuning
- Deploy
:::

```{figure} ./images/gpt-dev-cycle.png
---
width: 600px
name: gpt-dev-cycle
---
```

:::{admonition} An Idea
:class: tip
Identify clusters of people who are collaborating on Large Language Model research.
:::

```{figure} ./images/map-blank.png
---
width: 600px
name: map-blank
---
```

:::{admonition} Gather Data
:class: tip
Promising dataset: [arXive](https://arxiv.org/)
PDF files available, as well as [metadata](https://www.kaggle.com/datasets/Cornell-University/arxiv/)
:::

:::{admonition} Specify what information to extract
:class: tip
TBD
:::

:::{admonition} Develop a Prompt
[OpenAI playground](https://platform.openai.com/playground)
:::

:::{card} Prompt engineering
There are various prompting strategies you can use to improve performance. [OpenAI]([Prompt Engineering](https://platform.openai.com/docs/guides/prompt-engineering/strategy-write-clear-instructions) has a very good guide to help you out. They also provide lots of examples to look at.
:::

In [1]:
import os
import openai
from dotenv import load_dotenv

# load the .env file containing your API key
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

:::{admonition} Evaluate on a sample
:class: tip
TBD
:::

:::{admonition} Deploy
:class: tip
TBD
:::

:::{admonition} Payoff
:class: important
Map video
:::